1.실습: 삼성전자의 주가 데이터를 가져와서 분석하는 ChatGPT 어플리케이션을 작성하시오(힌트: pykrx 패키지)
실습: 사용자가 입력한 질문에 맞는 캠핑장을 검색해서 추천하는 ChatGPT 어플리케이션을 작성하시오(힌트: 고캠핑 API) -> 10~30분 필요 (가이드에 따름)

In [ ]:
import os
import openai
from pykrx import stock
import pandas as pd
from datetime import datetime

# ✅ OpenAI API 키 설정 (직접 입력하거나 환경변수에서 불러오기)
openai.api_key = os.getenv("OPENAI_API_KEY")  # 또는 직접 입력: "sk-..."

# 📌 삼성전자 주가 데이터 수집 함수
def get_samsung_stock(start_date: str, end_date: str) -> pd.DataFrame:
    df = stock.get_market_ohlcv_by_date(start_date, end_date, "005930")
    df = df.reset_index()
    return df

# 📊 데이터 요약 분석 함수
def summarize_stock_data(df: pd.DataFrame) -> str:
    start_price = df.iloc[0]["종가"]
    end_price = df.iloc[-1]["종가"]
    rate = ((end_price - start_price) / start_price) * 100

    summary = {
        "시작일": df.iloc[0]["날짜"].strftime('%Y-%m-%d'),
        "종료일": df.iloc[-1]["날짜"].strftime('%Y-%m-%d'),
        "시작 종가": start_price,
        "종료 종가": end_price,
        "최고가": df["고가"].max(),
        "최저가": df["저가"].min(),
        "평균 종가": round(df["종가"].mean(), 2),
        "수익률": round(rate, 2)
    }

    # ChatGPT용 텍스트 포맷팅
    summary_text = (
        f"{summary['시작일']}부터 {summary['종료일']}까지 삼성전자 주가 분석 결과입니다:\n"
        f"- 시작 종가: {summary['시작 종가']}원\n"
        f"- 종료 종가: {summary['종료 종가']}원\n"
        f"- 최고가: {summary['최고가']}원\n"
        f"- 최저가: {summary['최저가']}원\n"
        f"- 평균 종가: {summary['평균 종가']}원\n"
        f"- 총 수익률: {summary['수익률']}%\n"
    )
    return summary_text

# 🤖 ChatGPT 응답 생성 함수
def ask_chatgpt(summary_text: str, user_question: str) -> str:
    messages = [
        {"role": "system", "content": "You are a financial assistant that answers questions based on stock analysis."},
        {"role": "user", "content": f"다음은 삼성전자 주가 요약입니다:\n{summary_text}\n\n질문: {user_question}"}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        temperature=0,
        max_tokens=500
    )

    return response['choices'][0]['message']['content']

# 🧪 전체 실행 함수
def run_chatbot(start_date: str, end_date: str):
    df = get_samsung_stock(start_date, end_date)
    summary = summarize_stock_data(df)
    print("📊 분석 요약:\n", summary)

    # 사용자 질문 입력
    question = input("무엇이 궁금한가요? (예: 수익률이 좋은 편인가요?)\n> ")

    # ChatGPT에게 질문하기
    answer = ask_chatgpt(summary, question)
    print("\n🤖 ChatGPT의 답변:\n", answer)

# ✅ 실행 예시
if __name__ == "__main__":
    run_chatbot("20240101", "20240701")


📊 분석 요약:
 2024-01-02부터 2024-07-01까지 삼성전자 주가 분석 결과입니다:
- 시작 종가: 79600원
- 종료 종가: 81800원
- 최고가: 86000원
- 최저가: 70700원
- 평균 종가: 76892.62원
- 총 수익률: 2.76%



In [ ]:
# 데이터 생성

import os
import openai
from pykrx import stock

openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a financial assistant that answers questions based on stock analysis."},
    {
      "role": "user",
      "content": """

Q: [input]
A: [내가 한글로 입력한 input에 대한 대답만 하고, 너는 주가 분석 전문가야. 그리고 위에서 데이터를 기반으로만 답변을 해야해]
"""
    },
  ],
  temperature=0,
  max_tokens=1024,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response.choices[0].message.content)

In [1]:
import os
import openai
from pykrx import stock

# ✅ OpenAI API 키 설정
openai.api_key = os.getenv("OPENAI_API_KEY")

# ✅ 1. pykrx로 주가 데이터 수집 및 요약
def get_stock_summary(start_date: str, end_date: str) -> str:
    df = stock.get_market_ohlcv_by_date(start_date, end_date, "005930").reset_index()
    start_price = df.iloc[0]["종가"]
    end_price = df.iloc[-1]["종가"]
    rate = ((end_price - start_price) / start_price) * 100
    max_price = df["고가"].max()
    min_price = df["저가"].min()
    avg_price = round(df["종가"].mean(), 2)

    summary = (
        f"📈 삼성전자 주가 데이터 요약 ({start_date} ~ {end_date})\n"
        f"- 시작 종가: {start_price}원\n"
        f"- 종료 종가: {end_price}원\n"
        f"- 최고가: {max_price}원\n"
        f"- 최저가: {min_price}원\n"
        f"- 평균 종가: {avg_price}원\n"
        f"- 총 수익률: {rate:.2f}%\n"
    )
    return summary

# ✅ 2. 사용자 질문 및 데이터 기반 GPT 응답
def ask_question_based_on_data(user_question: str, stock_summary: str):
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": "You are a financial assistant that answers questions based on stock analysis data only."
            },
            {
                "role": "user",
                "content": f"""
아래는 삼성전자 주가 데이터입니다:

{stock_summary}

Q: {user_question}
A: [질문에 대한 한글 답변만 해줘. 주어진 데이터 외에는 답하지 마.]
"""
            },
        ],
        temperature=0,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].message.content

# ✅ 3. 실행
if __name__ == "__main__":
    summary = get_stock_summary("20250101", "20250721")
    print(summary)

    question = input("\n📨 질문을 입력하세요: ")
    answer = ask_question_based_on_data(question, summary)

    print("\n🤖 GPT의 답변:\n")
    print(answer)


📈 삼성전자 주가 데이터 요약 (20250101 ~ 20250721)
- 시작 종가: 53400원
- 종료 종가: 67800원
- 최고가: 68800원
- 최저가: 50800원
- 평균 종가: 57082.71원
- 총 수익률: 26.97%


🤖 GPT의 답변:

68800원


In [2]:
df = stock.get_market_ohlcv("20220720","20220810","005930")

In [3]:
df

,시가,고가,저가,종가,거래량,등락률
날짜,,,,,,
2022-07-20,61800,62100,60500,60500,16782238,-0.656814
2022-07-21,61100,61900,60700,61800,12291374,2.148760
2022-07-22,61800,62200,61200,61300,10261310,-0.809061
2022-07-25,60900,61900,60800,61100,9193681,-0.326264
2022-07-26,60800,61900,60800,61700,6597211,0.981997
2022-07-27,61300,61900,61200,61800,7320997,0.162075
2022-07-28,62300,62600,61600,61900,10745302,0.161812
2022-07-29,62400,62600,61300,61400,15093120,-0.807754
2022-08-01,61000,61700,60300,61300,13154816,-0.162866


In [ ]:
#데이터 프레임을 텍스트 형태로 변경해야함(llm에 주려면)
from to 